# WeatherPy


In [33]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [34]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

598

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [35]:
# Save config information
url = "http://api.openweathermap.org/data/2.5/weather?"

# Build query URL
query_url = url + "appid=" + weather_api_key 
query_url

'http://api.openweathermap.org/data/2.5/weather?appid=9d88a4e32284086ca01a89386c890bf8'

In [46]:
# set up lists to hold reponse info
city_names = []
longitudes = []
latitudes = []
max_temps = []
humidities = []
cloudiness1 = []
wind_speeds = []
countries = []
dates = []
#city_ids = []
count = 0

print(f"Beginning Data Retrieval: \n")
print(f"------------------------\n" )
# Loop through the list of cities and perform a request for data on each

for city in cities[:10]:
    city_url = query_url + "&q=" + city
    count += 1 
# Get weather data
    weather_response = requests.get(city_url)
    weather_json = weather_response.json()

# Try to grab the city name and id if they are available in the weather API
    try:
        
        city_name = weather_json['name']
        longitude = weather_json['coord']['lon']
        latitude = weather_json['coord']['lat']
        max_temp = weather_json['main']['temp_max']
        humidity = weather_json['main']['humidity']
        cloudiness = weather_json['clouds']['all']
        wind_speed = weather_json['wind']['speed']
        country = weather_json['sys']['country']
        date = weather_json['dt']
        #city_id = weather_json['id']
        
        city_names.append(city_name)
        longitudes.append(longitude)
        latitudes.append(latitude)
        max_temps.append(max_temp)
        humidities.append(humidity)
        cloudiness1.append(cloudiness)
        wind_speeds.append(wind_speed)
        countries.append(country)
        dates.append(date)
        #city_ids.append(city_id)

        print(f"Processing Record {count} | {city_name}")
        
# Handle exceptions for a city that is not available in the weather API
    except:
        # Append null values
        print(f"City not found. Skipping...")
        pass
 

Beginning Data Retrieval: 

------------------------

Processing Record 1 | Ushuaia
Processing Record 2 | Butaritari
Processing Record 3 | Inta
Processing Record 4 | Lere
Processing Record 5 | Hokitika
Processing Record 6 | Mataura
Processing Record 7 | Antofagasta
Processing Record 8 | Kapaa
Processing Record 9 | Panórama
Processing Record 10 | Punta Arenas


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [49]:
# create a data frame 
weather_dict = {
    "City": city_names,
    "Latitude": latitudes,
    "Longitude": longitudes,
    "Max Temp": max_temps,
    "Humidity": humidities,
    "Cloudiness": cloudiness1,
    "Wind Speed": wind_speeds,
    "Country": countries,
    "Date": dates
}
weather_data = pd.DataFrame(weather_dict)
weather_data

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ushuaia,-54.80,-68.30,274.15,80,40,8.70,AR,1594685734
1,Butaritari,3.07,172.79,300.39,74,85,0.53,KI,1594686027
2,Inta,66.03,60.17,283.34,75,71,4.24,RU,1594686027
3,Lere,10.39,8.57,294.24,93,98,2.37,NG,1594686027
4,Hokitika,-42.72,170.97,284.38,64,43,2.02,NZ,1594686027
5,Mataura,-46.19,168.86,277.04,83,91,1.34,NZ,1594685967
6,Antofagasta,-23.65,-70.40,286.15,76,0,3.10,CL,1594686028
7,Kapaa,22.08,-159.32,301.15,74,40,6.70,US,1594685842
8,Panórama,40.59,23.03,295.15,83,40,1.50,GR,1594686028
9,Punta Arenas,-53.15,-70.92,275.15,80,99,2.60,CL,1594685843


In [52]:
weather_data.to_csv (r'C:\Users\cdfen\Documents\Python-API-Challenge\Weatherpy\weatherdf.csv', index = False, header=True)

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


In [ ]:
# Extract relevant fields from the data frame


# Export the City_Data into a csv


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots

In [ ]:
# Create Northern and Southern Hemisphere DataFrames

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression